In [2]:
import cv2
import math
import numpy as np
import scipy.signal as scisig
import skimage.transform as sktr
from os import listdir
import skimage.io as skio
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')

In [3]:
# Read in the image in color or grayscale
def read_img(imname, path="data", gray=True):
    if gray:
        return cv2.imread(f'./{path}/{imname}', cv2.IMREAD_GRAYSCALE)
    else:
        return cv2.imread(f'./{path}/{imname}')
        #return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    
# Separate the image into rgb color channels
def get_color_channels(im):
    r = im[:,:,0]
    g = im[:,:,1]
    b = im[:,:,2]
    return r, g, b

# Display the image in notebook
def show_img(img, gray=True):
    if len(img.shape) == 3:
        plt.imshow(img[:,:,::-1])
    else: 
        plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.show()
    
# Saves the give images as jpg files
def save_img(imname, result):
    cv2.imwrite(f'./output/{imname}.jpg', result)

# Saves a list of images
def save_intermediate(imgs, imname):
    for i, im in enumerate(imgs):
        save_img(f'{imname}-{i}',auto_contrast(im))
 
# Contrast the image for better visualization
def auto_contrast(img):
    x, counts = np.unique(img, return_counts=True)
    cusum = np.cumsum(counts)
    pixel_mapping = {x[i]: cusum[i] / cusum[-1] * 255 for i in range(len(x))}
    convert = np.vectorize(lambda x: pixel_mapping[x])
    return convert(img).astype(np.uint8)

In [4]:
# Part 1.1: Finite Difference Operator

# Binarize the image
def binarize(im, threshold):
    return np.where(np.abs(im) > threshold, 255, 0)

# Convolve image with gradient filter 
def img_grad(im, thresh=60):
    dx = np.array([[1,-1], [0,0]])
    dy = np.array([[1,-1], [0,0]]).T
    
    im_x = scisig.convolve2d(dx, im, mode="valid")
    im_y = scisig.convolve2d(dy, im, mode="valid")

    bin_x = binarize(im_x, thresh)
    bin_y = binarize(im_y, thresh)
    
    return cv2.add(bin_x, bin_y)

im = read_img('cameraman.png')
result = img_grad(im)
show_img(result)
save_img('cameraman', result)

libpng warning: iCCP: profile 'icc': 0h: PCS illuminant is not D50


In [5]:
# Part 1.2: Derivative of Gaussian (DoG) Filter

# Blur image with Gaussian Kernel
def gaussian_blur(im, size=5, sigma=2, mode="valid"):
    kernel = cv2.getGaussianKernel(size, sigma)
    kernel_2d = kernel @ kernel.T
    return scisig.convolve2d(im, kernel_2d,  mode=mode)

im = read_img('cameraman.png')
blurred = gaussian_blur(im)
result = img_grad(blurred, thresh=20)

show_img(result)
save_img('cameraman_blurred', result)

libpng warning: iCCP: profile 'icc': 0h: PCS illuminant is not D50


In [127]:
# Part 2.1: Image "Sharpening"

# Extract high frequencies from image
def high_pass_filter(im, size=7, sigma=2):
    s = int(size/2)
    blurred = gaussian_blur(im, size, sigma)
    if size%2 == 0:
        im = im[s:len(im)-s, s:len(im)-s]
    else:
        im = im[s:len(im)-s, s:len(im[0])-s]
    high_freq = im - blurred
    return high_freq, im

# Sharpen image
def unsharp_mask_filter(im, alpha=6):
    high_freq, im = high_pass_filter(im)
    result = im + alpha * high_freq
    return result

imname = 'bellagio'
im = read_img(f'{imname}.jpeg', gray=False)
r,g,b = get_color_channels(im)

r_sharp = unsharp_mask_filter(r)
g_sharp = unsharp_mask_filter(g)
b_sharp = unsharp_mask_filter(b)

result = np.dstack([r_sharp, g_sharp, b_sharp])
save_img(f'sharp_{imname}', result)

In [239]:
# Try to reshapren already sharp image
im = read_img('landscape.jpeg', gray=False)

r,g,b = get_color_channels(im)

r_blur = gaussian_blur(r)
g_blur = gaussian_blur(g)
b_blur = gaussian_blur(b)

r_sharp = unsharp_mask_filter(r_blur)
g_sharp = unsharp_mask_filter(g_blur)
b_sharp = unsharp_mask_filter(b_blur)

result = np.dstack([r_sharp, g_sharp, b_sharp])
#save_img('resharpened', result)

In [31]:
# Part 2.2: Hybrid Images
# This cell contains all the given helper methods

def get_points(im1, im2):
    print('Please select 2 points in each image for alignment.')
    plt.imshow(im1)
    p1, p2 = plt.ginput(2)
    plt.close()
    plt.imshow(im2)
    p3, p4 = plt.ginput(2)
    plt.close()
    return (p1, p2, p3, p4)

def recenter(im, r, c):
    R, C = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(
        im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad),],
            'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    h1, w1 = im1.shape
    h2, w2 = im2.shape
    
    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2)
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2)
    dscale = len2/len1
    dscale_tuple = (1./dscale, 1./dscale)
    if dscale < 1:
        im1 = sktr.rescale(im1, dscale_tuple)
    else:
        im2 = sktr.rescale(im2, dscale_tuple)
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1 = im1.shape
    h2, w2 = im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : -int(np.ceil((h2-h1)/2.)), :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : -int(np.ceil((h1-h2)/2.)), :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : -int(np.ceil((w2-w1)/2.))]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : -int(np.ceil((w1-w2)/2.))]
    assert im1.shape == im2.shape
    return im1, im2

def align_images(im1, im2):
    pts = get_points(im1, im2)
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2

In [260]:
# Part 2.2: Continued

# Creates hybrid image
def create_hybrid(im1, im2, size1=30, sigma1=20, size2=31, sigma2=20):
    im1 = gaussian_blur(im1, size=size1, sigma=sigma1)
    im2 = high_pass_filter(im2, size=size2, sigma=sigma2)[0]
    im1, im2 = align_images(im2, im1)
    return im1 + im2

In [33]:
# Hybrid Image Example 1
im1 = read_img('DerekPicture.jpg', gray=True)
im2 = read_img('nutmeg.jpg', gray=True)
hybrid = create_hybrid(im1, im2)
hybrid[360:, :]
save_img('derek_nutemeg', hybrid)

Please select 2 points in each image for alignment.


In [261]:
# Hybrid Image Example 2
im1 = read_img('walter.jpeg', gray=True)
im2 = read_img('dog.jpeg', gray=True)
hybrid = create_hybrid(im1, im2)
save_img('walter_dog', hybrid)

Please select 2 points in each image for alignment.


In [106]:
# Hybrid Image Example 3
im1 = read_img('matt.jpeg', gray=True)
im2 = read_img('queen.jpeg', gray=True)
hybrid = create_hybrid(im1, im2, size1=15, size2=25)
save_img('matt_queen', hybrid)

Please select 2 points in each image for alignment.


In [ ]:
# Failed Hybrid Image
im1 = read_img('leo.jpeg', gray=True)
im2 = read_img('chris.jpeg', gray=True)
hybrid = create_hybrid(im1, im2, size1=15, size2=31)
save_img('leo_chris', hybrid)

In [224]:
# 2.2: Frequency Analysis
def get_freq_analysis(im):
    x = np.log(np.abs(np.fft.fftshift(np.fft.fft2(im))))
    x = auto_contrast(x)
    return x

im1 = read_img('walter.jpeg', gray=True)
im2 = read_img('dog.jpeg', gray=True)
hybrid = read_img('walter_dog.jpg')

#save_img('walter_freq', get_freq_analysis(im1))
#save_img('dog_freq', get_freq_analysis(im2))
#save_img('hybrid_freq', get_freq_analysis(hybrid))

In [106]:
# Part 2.3: Gaussian and Laplacian Stacks

# Convolves each rgb channel with a gaussian kernel
def convolve_colors(im, size=5, sigma=2):
    r = gaussian_blur(im[:,:,0], size=size, sigma=sigma, mode='same')
    g = gaussian_blur(im[:,:,1], size=size, sigma=sigma, mode='same')
    b = gaussian_blur(im[:,:,2], size=size, sigma=sigma, mode='same')
    res = np.dstack([r,g,b])
    return res

# Creates a gaussian stack for an image
def gaussian_stack(im, levels=5, size=30, sigma=1):
    gstack = [im]
    for i in range(levels):
        im_new = convolve_colors(gstack[i], size=size, sigma=sigma)
        gstack.append(im_new)
    return gstack

# Creates a laplacian stack for an image
def laplacian_stack(im):
    lstack = []
    gstack = gaussian_stack(im)
    for i in range(len(gstack)-1):
        lstack.append(gstack[i+1] - gstack[i])
    lstack.append(gstack[-1])
    return lstack

In [123]:
# Part 2.4: Multiresolution Blending

# Applies mask to laplacians and combines images
def blend_layers(la, lb, gr):
    ls = []
    for i in range(len(la)):
        blended =  gr[i] * la[i] + (1-gr[i]) * lb[i]
        ls.append(blended)
    return ls

# Blends two images together given a mask
def blend_img(im1, im2, mask):
    la = laplacian_stack(im1)
    lb = laplacian_stack(im2)
    gr = gaussian_stack(mask, size=50, sigma=15)
    ls = blend_layers(la, lb, gr)
    return np.sum(ls, axis=0)

In [124]:
# Drive code for Oraple
im1 = read_img('apple.jpeg', gray=False)
im2 = read_img('orange.jpeg', gray=False)

# Create mask of half 0s and half 1s
mask = np.zeros(im1.shape)
mask[:, :im1.shape[1]//2] = 1

result = blend_img(im1, im2, mask)
#save_img('oraple', result)

In [88]:
# Driver code for Custom Image 1
im1 = read_img('sathergate.jpeg', gray=False)
im2 = read_img('bear.jpeg', gray=False)
mask = read_img('bear_mask.png', gray=False)

# Pad the smaller image and mask
im2 = np.pad(im2, [(40,0), (225,225), (0,0)])
mask = np.pad(mask, [(40,0), (225, 225), (0,0)], constant_values=255)

# Set the mask values to 0 and 1
mask = np.where(mask >= 255, mask, 1)
mask = np.where(mask < 255, mask, 0)

# Blend the image and save
result = blend_img(im2, im1, mask)
#save_img('berkeley_bear', result)

In [121]:
# Driver code for Custom Image 2
im1 = read_img('cal_game.jpeg', gray=False)
im2 = read_img('great_dane.jpeg', gray=False)
mask = read_img('dane_mask.png', gray=False)

# Set the mask values to 0 and 1
mask = np.where(mask >= 255, mask, 1)
mask = np.where(mask < 255, mask, 0)

# Blend the image and save
result = blend_img(im2, im1, mask)
#save_img('dane_in_cal_game', result)